In [2]:
import numpy as np
import ndio.remote.neurodata as nd
from datetime import datetime

startTime = datetime.now()

oo = nd()

token = 'kasthuri2015_ramon_v1'
channel = 'synapses'
res = 3
pixel_dim = 0.024*0.024*0.030  #can get from LIMS

In [3]:
'''
There were 1,700 synapses at a density of one synapse per 1.13 μm3.

'''

import ndio.ramon as ramon

synapse_ids = oo.get_ramon_ids(token, channel, ramon_type=ramon.RAMONSynapse)

# A priori known bounds for cylinders.  Alternatively we could sweep over entire volume - this is more efficient.
# TODO:  assume that all synapses are inside cylinders, which we know to be true - should do with manual masking or a 
# RAMONId predicate query

xbox = [694,1794]; 
ybox = [1750, 2460];
zbox = [1004, 1379];

# These calls take about 60 seconds to execute
gcyl = oo.get_volume('kat11greencylinder','annotation', xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)
rcyl = oo.get_volume('kat11redcylinder','annotation', xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)
bcyl = oo.get_volume('kat11mojocylinder','annotation', xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)


mask = (gcyl.cutout + rcyl.cutout + bcyl.cutout) > 0

mask_pixels = sum(sum(sum(mask)))
mask_volume = mask_pixels * pixel_dim
print 'Mask Pixels: ' + str(mask_pixels)
print 'Mask Volume: ' + str(mask_volume) + ' um^3'

syn = oo.get_volume(token,channel, xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)
syn_volume = np.count_nonzero(syn.cutout)

print 'density of synapses is: {} / um^3'.format(round(1.0*(np.shape(synapse_ids)[0]) / mask_volume,4))
print 'fraction of volume that is psd: ' + str(round(1.0*syn_volume/mask_pixels,4))

print datetime.now() - startTime

Mask Pixels: 90152653
Mask Volume: 1557.83784384 um^3
density of synapses is: 1.0913 / um^3
fraction of volume that is psd: 0.0094
0:03:24.894439
